# CFD Domain's Creation & Discretization

**The next step for developing the CFD simulation for a case study, after the review of physics and theory behind the problem of interest, is to create the CFD/Computational domain. This CFD domain will first be created and then discretized into small finite elements (2D) or finite volumes (3D). The flow field's governing equations are then solved numerically inside these elements or volumes via an iterative process till a converged solution inside the CFD domain is obtained. This section explains the techniques and process of creation and discretization of the CFD domain for the problem of interest. It should be highlighted that this sections provides the most generic steps and techniques to create the CFD domain. Users can implement these steps/techniques in the mesher of their choice or download the pre-generated * .msh file from the "Simulation_Src" folder. The current tutorial provides methodological step-by-step  procedure to create the geometry of CFD domain and it's mesh in the official mesher package OpenFOAM, named blockMesh.**

The CFD domain consists of a rotating cylinder with diameter of 1 [m], located in the center, surrounded by a larger cylinder with 64 [m] diameter. The inner cylinder would rotate around its central axis and interact with the incoming flow. The outer cylinder forms the exterior bounds of the CFD domain. The dimension for the diameter of the inner cylinder is chosen to be 1 [m] to ease the calculation of non-dimensional variables such as Reynolds number and Drag Coefficient. The diameter of the larger cylinder is chosen to be 64 times the diameter of the rotating cylinder to remove any potential uncertainties on the physics of the flow field due to high blockage effect. A schematic of the CFD domain's raw geometry is shown in Fig. 1. As visualized in Fig. 1 the flow enters the CFD domain from the left half of the outer circle with a constant uniform velocity and exits the domain from the right half with a specific constant uniform pressure. 

<img src="./Images/1schematic.png" width="300">
Fig. 1. The schematic of the CFD domain raw geometry.

The `blockMesh` utility, as OpenFOAM's native hexahedral mesher, is used to create the raw geometry of the CFD domain and discretize (mesh) it. Using `blockMesh`, user can create and divide the CFD domain geometry into multiple blocks, define desired mesh resolutions for each block and produce a structural mesh. The `blockMesh` utility configurations and settings are stored in a text file named `blockMeshDict`, which is technically the "Meshing Dictionary" located in the 'system' folder of the working directory. In earlier versions of the OpenFOAM (lower than version 3.0.0) this dictionary was put in the `constant/polyMesh` directory. The OpenFOAM users start developing their own `blockMeshDict` file based on the previously developed files for similar or simpler case studies. The current `blockMeshDict` file is developed based on the original OpenFOAM tutorial `blockMeshDict` file named `cavity` located in `FOAM_Tutorials/incompressible/icoFoam/cavity/system`. The reason behind this choice was the fact that cavity tutorial has one of the most basic `blockMeshDict` file to create and mesh a single block. Therefore, it could be easily expanded to create and mesh the geometry for the current case study.

Taking advantage of the CFD domain's symmetry, only half of the geometry and it's mesh is created. Then using the mirror boolean operation, the full discretized CFD domain will be created. The half geometry is created using two separate quadrant blocks as shown in Fig. 2:

<img src="./Images/2blocks.png" width="500">
Fig. 2. Two blocks forming half of the CFD domain's raw geometry.

To create the two above-visualized blocks in Fig. 2 user should start editing the 'blockMeshDict' file. The first line in the `blockMeshDict` file, after the header section, is the `convertToMeters` command line. This command performs scaling process on the final produced geometry and it's mesh. Since the units of the current case study is [m], the set value for this command would be 1. If the units of the case study would be [mm] or [km] the scaling factor of 0.001 or 1000 would be set respectively to perform the scaling process correctly.

```C++
convertToMeters 1;
```

The next step is to create the two visualized blocks in Fig. 2 by defining their corner vertices as the skeleton of the CFD domain's geometry. This is done by entering the coordinate of each vertex under the `vertices` command line as shown below in the `blockMeshDict` file:

```C++
vertices
(
    (-0.5 0 0)
    (0 0.5 0)
    (0 32 0)
    (-32 0 0)
    (0.5 0 0)
    (32 0 0)

    (-0.5 0 1)
    (0 0.5 1)
    (0 32 1)
    (-32 0 1)
    (0.5 0 1)
    (32 0 1)
);
```

It is interesting to note that looking at these coordinates of these vertices reveals the fact that all of them are defined on two parallel xy planes with unit length offset. The reason behind this is that in OpenFOAM the concept of surface mesh is not defined! All mesh elements should be defined in 3D space. To model 2D case studies the `type` for boundaries in the third dimensions are defined as `empty`. More information and detail explanation on the boundary type can be read [here](http://cfd.direct/openfoam/user-guide/boundaries/).

The above-defined vertices are automatically labeled numerically by `blockMesh` in the order that are input into the `blockMesh` file. First input vertex would be labeled with number 0, the second vertex would be labeled with number 1 and this labeling process continues up to the last input vertex. These labeling process is visualized in Fig. 3.

<img src="./Images/3labels.png" width="500">
Fig. 3. Labels of the vertices forming the blocks of the domain.

At this stage that the skeleton of the CFD domain's geometry is defined, the two blocks are created by "connecting" vertices together and simultaneously define the meshing process of them using the `blocks` command line in the `blockMeshDict` file. Each block is defined using one `hex` command. The inputs of the `hex` command are a list of vertex labels, described above, a 3D vector defining the number of mesh elements in each direction, and the type and list of cell expansion ratio in each direction as shown below. In this case study the number of mesh is chosen to be 100 and 200 in angular and radial direction respectively. Furthermore, the total expansion ratio in the radial direction is set to 80 distributed using `simpleGrading` algorithm.

```C++
blocks
(
    hex (0 1 2 3 6 7 8 9) (100 200 1) simpleGrading (1 80 1)
    hex (1 4 5 2 7 10 11 8) (100 200 1) simpleGrading (1 80 1)
);
```
As you may have mentioned at this point the whole edges of the blocks are straight lines. If the mesh is produced up to this step, it should be look like this:

<img src="./Images/4straightEdges.png" width="500">
Fig. 4. The domain mesh without defining curved edges.

The next step is to define the curved edges of each block. There are four curved edges to define boarders of two circle quarters in two xy-planes for each block. The arc edges are defined using the `edges` command by their two end points stated using the labels of previously defined vertices and one axillary point, stated by it's coordinate, in the middle of the curved edge as follows:

```C++
edges
(
    arc 0 1 (-0.3535 0.3535 0)
    arc 2 3 (-22.6274 22.6274 0)
    arc 6 7 (-0.3535 0.3535 1)
    arc 8 9 (-22.6274 22.6274 1)

    arc 1 4 (0.3535 0.3535 0)
    arc 2 5 (22.6274 22.6274 0)
    arc 7 10 (0.3535 0.3535 1)
    arc 8 11 (22.6274 22.6274 1)
);
```

After defining the curved edges, the mesh would be look like this:

<img src="./Images/5halfMesh.png" width="500">
Fig. 5. The half CFD domain geometry and mesh after defining the curved edges.

In the final step of setting the `blockMeshDict` file, the location and type of the CFD domain boundaries should be set. The half domain has five boundaries including: inlet, outlet, cylinder, symm and frontAndBack which are shown in Fig. 6:

<img src="./Images/6BCs.png" width="500">
Fig. 6. Boundary conditions of the half domain .

Boundaries are face type geometries which are defined by the same vertices used in the blocking section. Each face is constructed by four vertices. A boundary can have one or more faces. In this section the basic boundary condition of each boundary is also defined. You can checkout the complete list of the basic boundary conditions in [here](http://cfd.direct/openfoam/user-guide/boundaries/). The vertices of a face are ordered as such that the normal vector of the face points to the inside of the domain.

```C++
boundary
(
    cylinder
    {
        type wall;
        faces
        (
            (1 0 6 7)
            (4 1 7 10)
        );
    }

    inlet
    {
        type patch;
        faces
        (
            (2 3 9 8)
        );
    }

    outlet
    {
        type patch;
        faces
        (
            (2 5 11 8)
        );
    }

    symm
    {
        type wall;
        faces
        (
            (0 3 9 6)
            (5 4 10 11)
        );
    }

    frontAndBack
    {
        type empty;
        faces
        (
            (1 2 3 0)
            (1 4 5 2)
            (6 9 8 7)
            (7 8 11 10)
        );
    }

);
```

At this stage the `blockMeshDict` file is completely setup for creating the mesh of half of the CFD domain. User should save the `blockMeshDict` file and compile it by running the following command in the working directory:

```bash
blockMesh
```

The output of compiling the created `blockmesh` file is visualized in Fig. 6. Each boundary condition is visualized by a different color.

<img src="./Images/7halfMeshBC.png" width="500">
Fig. 7. The half CFD domain geometry and mesh produced by the `blockMesh` utility.

Now a directory named `polyMesh` is produced in the `constant` directory in which the mesh is stored in OpenFOAM format. It contains multiple files including: boundary, faces, neighbour, owner and points. The most important file from all of these files is the boundary file which contains the boundary names and the basic boundary conditions of the domain specified earlier by the mesher tool which in here is the blockMesh utility.

```C++
5
(
    cylinder
    {
        type            wall;
        inGroups        1(wall);
        nFaces          50;
        startFace       19750;
    }
    inlet
    {
        type            patch;
        nFaces          25;
        startFace       19800;
    }
    outlet
    {
        type            patch;
        nFaces          25;
        startFace       19825;
    }
    symm
    {
        type            wall;
        inGroups        1(wall);
        nFaces          400;
        startFace       19850;
    }
    frontAndBack
    {
        type            empty;
        inGroups        1(empty);
        nFaces          20000;
        startFace       20250;
    }
)
```

Now it is time to finish the creation of CFD domain's geometry and mesh by using the mirror boolean operation. This step is done by executing `mirrorMesh` utility. `mirrorMesh` would reflect the existing geometry and mesh with respect to a defined plane of symmetry. The settings for this command are defined in the `mirrorMeshDict` file located in constant/polyMesh directory as follows:

```C++
planeType       pointAndNormal;

pointAndNormalDict
{
    basePoint       (0 0 0);
    normalVector    (0 -1 0);
}

planeTolerance  1e-06;
```

There are multiple types of defining a symmetry plane. Here the `pointAndNormal` type for specifying the symmetry plane is chosen. The plane is defined by an arbitrary point on the plane and its normal vector which are determined by `basepoint` and `normalVector` keywords respectively. The final step would be running the `mirrorMesh` command as follows:

```bash
mirrorMesh -overwrite
```

The `overwrite` flag in the command requires OpenFOAM to replace the previously saved mesh of the half domain mesh by the full domain mesh. Fig. 8 visualizes the full geometry and mesh created based on the created half domain and mesh using `mirrorMesh` command.

<img src="./Images/8finalMesh.png" width="500">
Fig. 8. The final mesh  created based on the created half domain using the mirrorMesh utility.

The `mirrorMesh` utility removes the internal boundaries after the reflection process for which in this case it would be the symm boundary. You may also notice this by looking at the `constant/polyMesh/boundary` file after the execution of the utility:

```C++
4
(
    cylinder
    {
        type            wall;
        inGroups        1(wall);
        nFaces          100;
        startFace       39900;
    }
    inlet
    {
        type            patch;
        nFaces          50;
        startFace       40000;
    }
    outlet
    {
        type            patch;
        nFaces          50;
        startFace       40050;
    }
    frontAndBack
    {
        type            empty;
        inGroups        1(empty);
        nFaces          40000;
        startFace       40100;
    }
)
```

Once the mesh is generated it's quality should be examined. This can be done using the provided features and functions defined in the meshing software. In general these functions calculate the aspect ratio of each elements and report a statistical report on them. The goal is to keep the average aspect ratio close to 1 for a very high quality mesh. In this stage the CFD domain is completely meshed and is ready to be setup for a CFD simulation.